# Introduction to PicoQuant

### About PicoQuant.

PicoQuant is a prototype of QuantEx: a platform consisting of modular quantum circuit simulation tools which use tensor network contractions methods. We use hierarchical layers of abstraction to encapsulate complexity and enable these tools to be easily extended and integrated into users’ circuit simulation codes. The layers are outlined as follows:


- Layer 1 - Operations in this layer perform the actual computations that are coordinated by
layer 2. This layer consists of a number of backends which are capable of
performing the computations for a variety of different platforms and scenarios.


- Layer 2 - Operations here are concerned with manipulating tensor network structures
and are responsible for coordinating and passing off to layer 1 functions
to perform the computations.


- Layer 3 - This is the highest layer and deals with the conversion of circuit
descriptions (QASM/qiskit circuit objects) to tensor networks. It provides
data structures and functionality to represent and manipulate tensor networks
representing quantum circuits.

### Simple example

Before explaining any of the workings we jump into a simple example which demonstrates the usage of PicoQuant using a 3-qubit GHZ state preparation circuit.

In [ ]:
using PicoQuant

tn = TensorNetworkCircuit(3)
add_input!(tn, "000")
add_gate!(tn, gate_tensor(:H), [1])
add_gate!(tn, gate_tensor(:CX), [1, 2])
add_gate!(tn, gate_tensor(:CX), [2, 3])
psi = full_wavefunction_contraction!(tn, "vector")

### Using PicoQuant

We now go through each of the lines in the above example and explain their workings. We then show the same example but by starting from a QASM circuit description.

Before doing anything we must import the PicoQuant package.

In [ ]:
using PicoQuant

We could also have import PicoQuant using the `import` keyword but would then need to prefix each function with `PicoQuant.`.

When creating a tensor network, we can choose to initialise a backend. By default, an InteractiveBackend is used. Backends are responsible for keeping track of any tensors created/removed during the creation and contraction of tensor networks. When the user calls a function that alters the structure of a tensor network, the function will use the given backend to invoke function methods that handle the tensor data in a way that is specific to that backend. We currently have two different backends that may be used:

1. Interactive backend:
    - This backend is initialised by passing InteractiveBackend() to the TensorNetworkCircuit constructor. It may also be omitted as it is the default backend if one is not specified.
    - This backend contains a dictionary of all the tensors in the tensor network
    - When a function is called to carry out an operation (contraction, reshape, decompose, permute etc...) on tensors in the tensor network, it is carried out immediately and the tensors are updated in realtime  
    
2. DSL (Domain Specific Language) backend:
    - This backend is initialised by passing a DSLBackend("dsl_file.tl", "tensor_file.h5", "output_file.h5") to the TensorNetworkCircuit constructor.
    - When tensors/gates are added to a tensor network, this backend will write the tensor data to the given hdf5 file for storing tensors. Tensors that are the result of contracting a network will be written to the given output file. By default, if no arguments are given to this constructor, both tensor and output data will be written to a file called "tensor_data.h5"
    - When a function is called to carry out an operation (contraction, reshape, decompose, permute etc...) on tensors in the tensor network, the DSL command describing this operation is written to the DSL file for later execution     

For the rest of this notebook we will use the InteractiveBackend. In notebook 3 we will demonstate the usage of the DSL backend.

In [ ]:
# Create an instance of TensorNetworkCircuit. 
# The integer argument tells the constructor how many qubits we would like to have in our circuit.
# As the InteractiveBackend is the default, we can omit it here.
tn = TensorNetworkCircuit(3)

The output of the above cell shows the fields contained in our TensorNetworkCircuit instance. More details about the data structure used by PicoQuant to represent tensor networks are given in the notebook titled 'Tensor-network-data-structure'.

Now, we would like to add some gates to our circuit. We can do this by calling the add_gate! function. The arguments for this function are: 
1. a TensorNetworkCircuit to add a gate to, 
2. a tensor containing the gate data and 
3. an array of integers identifying the target qubits of the gate. 

PicoQuant also provides a gate_tensor() function which returns the tensor data for some commonly used quantum gates. We can get a list of accepted input for gate_tensor as follows: 

In [ ]:
?gate_tensor

Below, we create tensor data for a Hadamard gate and a CNOT gate and then add these to our circuit to create a 3-qubit GHZ circuit. 

In [ ]:
# Create tensors for the hadamard and CNOT gates
# (Note, as the CNOT gate acts on two qubits we require the gate data to be contained in a rank 4 tensor.)
hadamard_data = gate_tensor(:H)
CNOT_data = gate_tensor(:CX)

# Add the gates to the circuit in the desired order.
# Note for adding the controlled not gates, the array identifying the qubits to act on is of the form
# [target_quibt, controll_qubit]
add_gate!(tn, hadamard_data, [1])
add_gate!(tn, CNOT_data, [1, 2])
add_gate!(tn, CNOT_data, [2, 3])

We can specify an initial state for the input qubits using the add_input! function. To call it, we need to pass it our TensorNetworkCircuit instance and a string of 0's and 1's indicating a computational basis state to put our input qubits in. This function then adds a node to the TensorNetworkCircuit for each input qubit. The added nodes will contain the appropriate single qubit state vector specified by the given string. Below, we add nodes to our TensorNetworkCircuit for input qubits in the state $|000>$. 

In [ ]:
add_input!(tn, "000")

To visually inspect the tensor network we have created we can plot it using the PicoQuant plot function.

In [ ]:
# Note, the edges representing the open indices of the network are not shown.
plot(tn)

To get the output of the circuit we need to contract the tensor network. We can do this by calling the full_wavefunction_contraction! function. This will first contract the nodes corresponding to the input qubits to get a tensor representing the wavefunction of the input qubits. It then contracts the gates of the circuit into the wavefunction tensor in the same order they appear in the circuit. We can also pass the full_wavefunction_contraction! function an optional argument specifying the shape the resulting tensor. This optional argument can either be an array of integers giving the shape of the final tensor or it can be the string "vector" which will tell the function to rehsape the final tensor into a vector.

In [ ]:
full_wavefunction_contraction!(tn, "vector")

With the interactive backend, the full_wavefunction_contraction! function will return the output of the circuit. The output will also be saved by the backend under the name :result. We can access the saved output by using the load_tensor_data function, which returns the data associated with a given node label, as follows:

In [ ]:
circuit_output = load_tensor_data(tn, :result)

It may happen that we are only interested in the output amplitude for a specific computational basis state, rather than the entire output state. If that is the case, we can also add nodes to the tensor network to represent the basis state of interest. Then, contracting the network will return the inner product of the circuits' output state and the given basis state. Adding output nodes is similar to adding input nodes: We call the function 'add_output!' which takes similar arguments to 'add_input!'

## Qiskit and Qasm

PicoQuant also provides functions for creating a qiskit circuit object from a qasm description of a quantum circuit and converting that circuit object to a TensorNetworkCircuit. We demonstrate the use of these functions below.

We start by creating a qasm description of a simple quantum circuit and then use it to gernerate a qiskit circuit object.

In [ ]:
# A qasm description of the 3 qubit GHZ circuit.
qasm_str = """OPENQASM 2.0;
              include "qelib1.inc";
              qreg q[3];
              h q[0];
              cx q[0],q[1];
              cx q[1],q[2];"""

# We can create a qiskit circuit object for this circuit by passing the above string to the following function.
circ = load_qasm_as_circuit(qasm_str)

There is also a 'load_qasm_as_circuit_from_file' to generate a qiskit circuit object from a file containing qasm. To call it, we need only to pass it a string with the path to the qasm file.

Once a qiskit circuit object is created, we may call any of the methods contained in the object to add gates or barriers to the circuit. Below we add a barrier to the out circuit followed by a couple of controlled-$Z$ gates before drawing the circuit.

In [ ]:
circ.barrier()
circ.cz(1, [0,2])
circ.draw()

We can use this circuit object to a generate a TensorNetworkCircuit instance for the same circuit by passing it to the 'convert_qiskit_circ_to_network' function. This function will create a TensorNetworkCircuit instance with the same number of qubits and then add the same gates to it in the same order as in the circuit object. Any barriers added to the qiskit circuit will be ignored and do not appear in the TensorNetworkCircuit.

In [ ]:
# This function creates the TensorNetworkCircuit from the qiskit circuit.
# Again we choose an InteractiveBackend and show the explicit usage here.
tn = convert_qiskit_circ_to_network(circ, InteractiveBackend())

# Here we add the basis state |000> as the initial state of the qubits in the circuit,
# and add the vector |111> to the end of the circuit.
add_input!(tn, "000")
add_output!(tn, "111")

# Contracting the network should now produce the amplitude of the circuit's output state corresponding 
# to the state |111>
output_node = full_wavefunction_contraction!(tn)
load_tensor_data(tn, output_node)

### Using qiskit

In the above we are using qiskit to read the QASM file and convert the gates to matrices. We can also use qiskit directly to construct circuits. For this we use the PyCall to import the qiskit module. For example to construct the same circuit as above we would use the following (note the 0-based indexing of qubits)

In [ ]:
using PyCall

qiskit = pyimport("qiskit")
circ = qiskit.QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.draw()

### Short Exercise
The following is a short exercise to help get familiar with functions introduced above. The aim is that given the GHZ preparation circuit above, add additional gates so that the output state is again the all zero state $|000\rangle$.

In [ ]:
tn = TensorNetworkCircuit(3)
add_input!(tn, "000")

hadamard_data = gate_tensor(:H)
CNOT_data = gate_tensor(:CX)
add_gate!(tn, hadamard_data, [1])
add_gate!(tn, CNOT_data, [1, 2])
add_gate!(tn, CNOT_data, [2, 3])

Enter you code in the following cell to add gates to return the state to all zeros

In [ ]:
# enter you code here to add gates to return the state to 000 state


We contract the network and check the final state is indeed the all zero state.

In [ ]:
full_wavefunction_contraction!(tn, "vector")
if load_tensor_data(tn, :result) ≈ kron([1, 0], [1, 0], [1, 0]) 
    println("Correct")
else
    println("State does not match expected answer")
end

### Area of Application

Up until recently it has been possible to simulate the largest prototype universal quantum computers using direct evolution of the quantum state (where the full wave-function is stored in memory (or on disk)). This is a very computationally demanding problem owing to the exponential growth in the state space as the number of qubits is increased. With the recent emergence of Noisy Intermediate Scale Quantum (NISQ) devices, it has become intractable to use this approach to simulate devices of this size on even the largest supercomputers.


The most promising alternative to direct evolution of the full wave-function are tensor network contraction methods. They work by representing quantum states and operators using networks of tensors where calculations correspond to contractions over network edges. Tensor network methods are also approximate methods, well approximating many qubit states with low entanglement. Thus, while a full wave function simulator is ideal for simulating quantum circuits with a small number of qubits, tools such as QuantEx will be most beneficial for simulating quantum circuits with low depth and high width. This is illustrated in the graphic below:


![title](img/area_of_application.svg)

### Exercise solution

To undo the operation one can apply the inverse of the circuit by adding the gates in reverse order.

In [ ]:
add_gate!(tn, CNOT_data, [2, 3])
add_gate!(tn, CNOT_data, [1, 2])
add_gate!(tn, hadamard_data, [1])